# Rescale RGB image for spatial chunks

This notebook shows a simple process for rescaling Sentinel 2 RGB images within polygon chunks that also showcases how to use ``chunk_polygon()`` with a (User Defined Function) UDF. (To be noted: chunk_polygon are experimental at the moment)

In [ ]:
# import necessary packages
import openeo
from openeo.api.process import Parameter
import json
from pathlib import Path
import matplotlib.pyplot as plt
import rasterio

# connect with the backend
eoconn = openeo.connect("openeo.vito.be").authenticate_oidc()

Authenticated using refresh token.


User can choose among different backend available [here](https://hub.openeo.org/) to connect to the backend of their choice. Regarding the authentication process OpenID connect (oidc) is recommended, but not always straightforward to use. In cases where you are unable to connect with the backend use basic authentication method explained [here](https://openeo.org/documentation/1.0/authentication.html#openid-connect).

In [2]:
# function to load geojson file
def read_json(path: Path) -> dict:
    with open(path) as input:
        field = json.load(input)
        input.close()
    return field

To use the data collection, a user must use the correct backend with the data collection. Then using load_collection, they can specify bands, temporal extent (i.e. interested time interval) and even spatial extent. In this example, we have loaded the entire collection so that process (including UDF) can later be applied to spatial chunks. 

In [3]:
# Load your data cube based on your prefernce

S2_cube = eoconn.load_collection(
    "SENTINEL2_L2A",
    temporal_extent = ["2022-06-04", "2022-08-04"],
    bands = ["B02", "B03", "B04"]
)

Here we tried in presenting a method to create and use UDF as an openEO feature. In a similar manner user can create their own UDF as needed to apply to their data cube. More information on [UDF](https://open-eo.github.io/openeo-python-client/udf.html).

In [4]:
# Create a UDF object from inline source code.
my_udf = openeo.UDF("""
from openeo.udf import XarrayDataCube

def apply_datacube(cube: XarrayDataCube, context: dict) -> XarrayDataCube:
    array = cube.get_array()
    array.values = 0.0001 * array.values
    return cube
""")

We used the chunk_polygon method to apply our UDF over a spatial chunk of the datacube. In the case of a simple process that does not require UDF, you can directly load your spatial extent in the dataset. 

Furthermore, since we loaded our collection for specific time intervals, it can include multiple time dimensions. Thus [reduce_dimension](https://processes.openeo.org/#reduce_dimension) applies a reducer to a data cube dimension by collapsing all the pixel values along the time dimension into an output value computed by the reducer.

In [ ]:
# apply rescale to chunks of polygon
aoi = read_json("cologne_aoi.geojson")
rescaled_chunks = S2_cube.chunk_polygon(chunks=aoi,process=my_udf)

# perform time dimension reduction
Rrescaled_chunks = rescaled_chunks.reduce_dimension(dimension="t", reducer="mean")

Once the process is completed, you can also save it as your process using [save_user_defined_process](https://open-eo.github.io/openeo-python-client/udp.html) that can later be used for a similar task. Otherwise, you can download the result either by direct download (in case of the small spatial extent with few processing) or perform create a [batch job](https://open-eo.github.io/openeo-python-client/batch_jobs.html) in case it is a heavy task over a large extent.

In [8]:
## download your result either using synchronous method or batch
# synchronous download
# rescaled_chunks.download("rescaled_test_v1.tiff")
# 
# Or perform batch processing if area is comparatively large
batch_job = Rrescaled_chunks.create_job(out_format = "GTiff", title="rescaled_chunks2")
batch_job.start_and_wait()
results = batch_job.get_results()
results.download_files()